# An Important Note
I worked on this project during my studies for Dataquest online Data Science Bootcamp. This was for "Pandas and NumPy Fundamentals" part of the bootcamp.

# Analyzing Used Car Listings on eBay Kleinanzeigen
I will be working on a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

The dataset was originally scraped and uploaded to Kaggle. The version of the dataset I am working with is a sample of 50,000 data points that was prepared by Dataquest including simulating a less-cleaned version of the data.

The aim of this project is to clean the data and analyze the included used car listings.

# Reading The Data Into Pandas

In [1]:
import pandas as pd
import numpy as np
autos = pd.read_csv("autos.csv", encoding="Latin-1")

In [2]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


From the work I did above, I can make the following observations:

1. The dataset contains 20 columns, most of which are strings.
2. Some columns have null values, but none have more than ~20% null values.
3. The column names use camelcase instead of Python's preferred snakecase, which means I can't just replace spaces with underscores.

<br/>I will start by converting the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to be more descriptive.

In [3]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

I'll make a few changes here:

1. Change the columns from camelcase to snakecase.
2. Change a few wordings to more accurately describe the columns.

In [4]:
autos.rename({'yearOfRegistration': 'registration_year'}, axis=1, inplace= True)
autos.rename({'monthOfRegistration': 'registration_month'}, axis=1, inplace= True)
autos.rename({'notRepairedDamage': 'unrepaired_damage'}, axis=1, inplace= True)
autos.rename({'dateCreated': 'ad_created'}, axis=1, inplace= True)
autos.rename({'dateCrawled': 'date_crawled'}, axis=1, inplace= True)
autos.rename({'offerType': 'offer_type'}, axis=1, inplace= True)
autos.rename({'vehicleType': 'vehicle_type'}, axis=1, inplace= True)
autos.rename({'lastSeen': 'last_seen'}, axis=1, inplace= True)

In [5]:
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'powerPS', 'model',
       'odometer', 'registration_month', 'fuelType', 'brand',
       'unrepaired_damage', 'ad_created', 'nrOfPictures', 'postalCode',
       'last_seen'],
      dtype='object')

As it can be seen, the column names are changed.

# Initial Data Exploration and Cleaning
I'll start by exploring the data to find obvious areas where I can clean the data.

In [6]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,powerPS,model,odometer,registration_month,fuelType,brand,unrepaired_damage,ad_created,nrOfPictures,postalCode,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-30 17:37:35,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


My initial observations:

1. There are two of text columns where almost all of the values are the same:<br/>
     seller<br/>
     offer_type
2. The "nrOfPictures" column has only "0" values.

So, I will start by dropping these three columns.

In [8]:
autos = autos.drop(["nrOfPictures", "seller", "offer_type"], axis=1)

In [10]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 17 columns):
date_crawled          50000 non-null object
name                  50000 non-null object
price                 50000 non-null object
abtest                50000 non-null object
vehicle_type          44905 non-null object
registration_year     50000 non-null int64
gearbox               47320 non-null object
powerPS               50000 non-null int64
model                 47242 non-null object
odometer              50000 non-null object
registration_month    50000 non-null int64
fuelType              45518 non-null object
brand                 50000 non-null object
unrepaired_damage     40171 non-null object
ad_created            50000 non-null object
postalCode            50000 non-null int64
last_seen             50000 non-null object
dtypes: int64(4), object(13)
memory usage: 6.5+ MB


,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,powerPS,model,odometer,registration_month,fuelType,brand,unrepaired_damage,ad_created,postalCode,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50


As it can be seen, those three columns are dropped. But now, there is a an other problem in the data set. There are two columns "price" and "odometer" which are supposed to have numerical values but because of the extra characters, these two columns are stored as text. I'll continue cleaning of the data by removing the extra characters from these two columns.

In [11]:
autos["price"] = autos["price"].str.replace('$','')
autos["price"] = autos["price"].str.replace(',','')
autos["price"] = autos["price"].astype(float)

In [12]:
autos["price"].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

The "price" column is cleaned now and has numerical values.

In [13]:
autos["odometer"] = autos["odometer"].str.replace('km','')
autos["odometer"] = autos["odometer"].str.replace(',','')
autos["odometer"] = autos["odometer"].astype(int)

In [14]:
autos["odometer"].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer, dtype: float64

The "odometer" column is cleaned now and has numerical values. Before the cleaning of this column, it was easily understandable that the distances are in kilometres but now I must indicate that specially. That's why, I'll change the column name to "odometer_km".

In [15]:
autos.rename({'odometer':'odometer_km'}, axis=1, inplace=True)

In [16]:
autos.columns

Index(['date_crawled', 'name', 'price', 'abtest', 'vehicle_type',
       'registration_year', 'gearbox', 'powerPS', 'model', 'odometer_km',
       'registration_month', 'fuelType', 'brand', 'unrepaired_damage',
       'ad_created', 'postalCode', 'last_seen'],
      dtype='object')

# Exploring Price and Odometer Informations

In [17]:
autos["price"].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

When I describe the "price" column, I see that the maximum value is 10000000 dollars and the minimum value is 0 dollars which are nor acceptable. So,I need to remove these kind of datas from the data set.

In [18]:
sorted_price = autos["price"].sort_index(ascending=False)
sorted_price.describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [19]:
autos["price"] = sorted_price
autos["price"].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

At this point, I decided to remove the rows (cars) with a price more than 100000 USD and less than 1000 USD.

In [20]:
autos.drop(autos[(autos["price"] > 100000)].index, inplace=True)

In [21]:
autos.drop(autos[(autos["price"] < 1000)].index, inplace=True)

In [22]:
autos.describe()

,price,registration_year,powerPS,odometer_km,registration_month,postalCode
count,38587.000000,38587.000000,38587.000000,38587.000000,38587.000000,38587.000000
mean,7089.045482,2005.680125,128.663177,122859.771426,5.999559,51682.914298
std,7893.814349,86.727780,215.791791,40712.638070,3.591692,25689.243673
min,1000.000000,1000.000000,0.000000,5000.000000,0.000000,1067.000000
25%,2200.000000,2001.000000,80.000000,100000.000000,3.000000,31199.000000
50%,4300.000000,2005.000000,116.000000,150000.000000,6.000000,50829.000000
75%,8950.000000,2009.000000,160.000000,150000.000000,9.000000,72528.000000
max,99900.000000,9999.000000,17700.000000,150000.000000,12.000000,99998.000000


I have removed the prices which are greater than 100000 dollars and less than 1000 dollars. At the end we have 38587 rows (cars) instead of 50000.

In [23]:
autos["odometer_km"].describe()

count     38587.000000
mean     122859.771426
std       40712.638070
min        5000.000000
25%      100000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

As we can see there is no information which is not normal under odometer_km column.

# Exploring the date columns
There are a number of columns with date information:

1. date_crawled
2. registration_month
3. registration_year
4. ad_created
5. last_seen<br/>

These are a combination of dates that were crawled, and dates with meta-information from the crawler. The non-registration dates are stored as strings.

I'll explore each of these columns to learn more about the listings.

In [24]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [26]:
autos["date_crawled"].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025527
2016-03-06    0.013891
2016-03-07    0.035167
2016-03-08    0.032602
2016-03-09    0.032472
2016-03-10    0.033353
2016-03-11    0.032835
2016-03-12    0.037396
2016-03-13    0.016016
2016-03-14    0.036670
2016-03-15    0.033612
2016-03-16    0.029077
2016-03-17    0.030477
2016-03-18    0.012802
2016-03-19    0.035167
2016-03-20    0.038173
2016-03-21    0.037266
2016-03-22    0.032420
2016-03-23    0.032213
2016-03-24    0.029025
2016-03-25    0.030528
2016-03-26    0.033094
2016-03-27    0.031384
2016-03-28    0.035349
2016-03-29    0.034001
2016-03-30    0.033016
2016-03-31    0.031410
2016-04-01    0.034571
2016-04-02    0.036333
2016-04-03    0.039132
2016-04-04    0.036904
2016-04-05    0.013346
2016-04-06    0.003265
2016-04-07    0.001503
Name: date_crawled, dtype: float64

It looks like the site was crawled daily over roughly a one month period in March and April 2016. The distribution of listings crawled on each day is roughly uniform.

In [27]:
autos["ad_created"].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2015-06-11    0.000026
2015-08-10    0.000026
2015-09-09    0.000026
2015-11-10    0.000026
2015-12-30    0.000026
2016-01-03    0.000026
2016-01-07    0.000026
2016-01-10    0.000052
2016-01-13    0.000026
2016-01-14    0.000026
2016-01-16    0.000026
2016-01-22    0.000026
2016-01-27    0.000078
2016-01-29    0.000026
2016-02-01    0.000026
2016-02-02    0.000052
2016-02-05    0.000052
2016-02-07    0.000026
2016-02-09    0.000026
2016-02-11    0.000026
2016-02-12    0.000052
2016-02-14    0.000052
2016-02-16    0.000026
2016-02-17    0.000026
2016-02-18    0.000052
2016-02-19    0.000078
2016-02-20    0.000026
2016-02-21    0.000052
2016-02-22    0.000026
2016-02-23    0.000104
                ...   
2016-03-09    0.032679
2016-03-10    0.033016
2016-03-11    0.033094
2016-03-12    0.037137
2016-03-13    0.017674
2016-03-14    0.035012
2016-03-15    0.033431
2016-03-16    0.029647
2016-03-17    0.030140
2016-03-18    0.013269
2016-03-19    0.034105
2016-03-20    0.038277
2016-03-21 

There is a large variety of ad created dates. Most fall within 1-2 months of the listing date, but a few are quite old, with the oldest at around 9 months.

In [28]:
autos["last_seen"].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05 14:46:02    0.000026
2016-03-05 14:49:34    0.000026
2016-03-05 15:16:11    0.000026
2016-03-05 15:16:47    0.000026
2016-03-05 15:28:10    0.000026
2016-03-05 15:41:30    0.000026
2016-03-05 15:45:43    0.000026
2016-03-05 15:47:38    0.000026
2016-03-05 15:47:44    0.000026
2016-03-05 16:45:57    0.000026
2016-03-05 17:15:45    0.000026
2016-03-05 17:16:14    0.000026
2016-03-05 17:17:02    0.000026
2016-03-05 17:39:19    0.000026
2016-03-05 17:40:14    0.000026
2016-03-05 17:44:50    0.000026
2016-03-05 17:44:54    0.000026
2016-03-05 18:17:58    0.000026
2016-03-05 18:50:38    0.000026
2016-03-05 19:15:08    0.000026
2016-03-05 19:15:42    0.000026
2016-03-05 19:16:36    0.000026
2016-03-05 19:17:17    0.000026
2016-03-05 19:17:50    0.000026
2016-03-05 19:32:34    0.000026
2016-03-05 19:45:47    0.000026
2016-03-05 19:46:20    0.000026
2016-03-05 20:15:42    0.000026
2016-03-05 20:15:53    0.000026
2016-03-05 20:16:33    0.000026
                         ...   
2016-04-

The crawler recorded the date it last saw any listing, which allows us to determine on what day a listing was removed, presumably because the car was sold.

# Dealing with Incorrect Registration Year

In [29]:
autos["registration_year"].describe()

count    38587.000000
mean      2005.680125
std         86.727780
min       1000.000000
25%       2001.000000
50%       2005.000000
75%       2009.000000
max       9999.000000
Name: registration_year, dtype: float64

Since, the maximum value is 9999 and the minimum value is 1000, these informations are not acceptable. I have to remove these kind of informations which are not normal. In my project I will set the minimum registration year to 1950 and maximum registration year to 2016.

In [30]:
autos.drop(autos[(autos["registration_year"] > 2016)].index, inplace=True)

In [31]:
autos.drop(autos[(autos["registration_year"] < 1950)].index, inplace=True)

In [32]:
autos.describe(include='all')

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,powerPS,model,odometer_km,registration_month,fuelType,brand,unrepaired_damage,ad_created,postalCode,last_seen
count,37152,37152,37152.000000,37152,35815,37152.000000,35885,37152.000000,35730,37152.000000,37152.000000,35322,37152,31975,37152,37152.000000,37152
unique,36157,28419,NaN,2,8,NaN,2,NaN,241,NaN,NaN,7,40,2,72,NaN,30154
top,2016-03-29 23:42:13,BMW_318i,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,2016-04-07 03:16:17
freq,3,67,NaN,19175,10542,NaN,26761,NaN,2918,NaN,NaN,21290,7842,29661,1469,NaN,7
mean,NaN,NaN,7177.566214,NaN,NaN,2003.994644,NaN,129.263889,NaN,122660.556632,6.044170,NaN,NaN,NaN,NaN,51822.354436,NaN
std,NaN,NaN,7959.881625,NaN,NaN,6.877906,NaN,197.293911,NaN,40760.776953,3.575337,NaN,NaN,NaN,NaN,25679.581491,NaN
min,NaN,NaN,1000.000000,NaN,NaN,1950.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,1067.000000,NaN
25%,NaN,NaN,2222.000000,NaN,NaN,2001.000000,NaN,82.000000,NaN,100000.000000,3.000000,NaN,NaN,NaN,NaN,31275.000000,NaN
50%,NaN,NaN,4444.000000,NaN,NaN,2005.000000,NaN,117.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,51063.000000,NaN
75%,NaN,NaN,8999.000000,NaN,NaN,2008.000000,NaN,160.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,72624.250000,NaN


In [33]:
autos["registration_year"].value_counts(normalize=True)

2005    0.074962
2006    0.071356
2004    0.070198
2003    0.066672
2007    0.060777
2008    0.059270
2002    0.057467
2009    0.055879
2001    0.055556
2000    0.053833
1999    0.046404
2011    0.043497
2010    0.042528
2012    0.035072
1998    0.034453
2013    0.021291
1997    0.021156
2014    0.017415
2016    0.017361
1996    0.014589
1995    0.011897
2015    0.009609
1994    0.007294
1993    0.005868
1992    0.005841
1991    0.005572
1990    0.005087
1989    0.003311
1988    0.002826
1985    0.002126
          ...   
1970    0.000861
1979    0.000834
1972    0.000807
1981    0.000700
1968    0.000673
1967    0.000673
1971    0.000646
1973    0.000619
1974    0.000619
1966    0.000565
1976    0.000565
1977    0.000511
1969    0.000484
1975    0.000484
1965    0.000458
1960    0.000458
1964    0.000242
1963    0.000215
1959    0.000161
1961    0.000161
1956    0.000108
1962    0.000108
1958    0.000081
1957    0.000054
1955    0.000054
1954    0.000054
1953    0.000027
1950    0.0000

After removing the anormal informations I have total of 37152 rows in the data set.

# Exploring Price by Brand

In [35]:
autos["brand"].value_counts(ascending=False, normalize=True)

volkswagen        0.211079
bmw               0.125431
mercedes_benz     0.111623
audi              0.097707
opel              0.089174
ford              0.058678
renault           0.037333
peugeot           0.027939
fiat              0.021102
skoda             0.019084
seat              0.017307
smart             0.016634
toyota            0.014643
mazda             0.014266
citroen           0.013916
nissan            0.013647
mini              0.010901
hyundai           0.010767
sonstige_autos    0.010148
volvo             0.008990
kia               0.007698
honda             0.007348
mitsubishi        0.006891
porsche           0.006783
chevrolet         0.006621
alfa_romeo        0.006245
suzuki            0.005733
dacia             0.003284
chrysler          0.003176
jeep              0.002772
land_rover        0.002638
jaguar            0.001857
subaru            0.001723
daihatsu          0.001696
saab              0.001373
daewoo            0.000915
trabant           0.000861
r

Volkswagen is by far the most popular brand, with approximately double the cars for sale of the next two brands combined.<br/>
There are lots of brands that don't have a significant percentage of listings, so we will limit our analysis to brands representing more than 5% of total listings.

In [40]:
brand_counts = autos["brand"].value_counts(normalize=True)
highest_brands = brand_counts[brand_counts > .05].index
print(highest_brands)

Index(['volkswagen', 'bmw', 'mercedes_benz', 'audi', 'opel', 'ford'], dtype='object')


In [42]:
mean_price_for_highest_brands = {}
for brand in highest_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_price = brand_only["price"].mean()
    mean_price_for_highest_brands[brand] = int(mean_price)
print(mean_price_for_highest_brands)

{'audi': 10276, 'volkswagen': 6640, 'bmw': 8974, 'opel': 4220, 'mercedes_benz': 9192, 'ford': 5258}


For the prices of these 6 brands of used cars, I can say that;
1. BMW, Mercedes and Audi are more expensive,
2. Opel and Ford are more cheaper,
3. Volkswagen is in between.

# Exploring Mileage

In [43]:
mpfhb_series = pd.Series(mean_price_for_highest_brands)
pd.DataFrame(mpfhb_series, columns=["mean_price"])

,mean_price
audi,10276
bmw,8974
ford,5258
mercedes_benz,9192
opel,4220
volkswagen,6640


In [44]:
mean_miles_for_highest_brands = {}
for brand in highest_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_mileage = brand_only["odometer_km"].mean()
    mean_miles_for_highest_brands[brand] = int(mean_mileage)

mean_mileage = pd.Series(mean_miles_for_highest_brands).sort_values(ascending=False)
mean_prices = pd.Series(mean_price_for_highest_brands).sort_values(ascending=False)

In [46]:
highest_brand_ds = pd.DataFrame(mean_mileage,columns=['mean_mileage'])
highest_brand_ds

,mean_mileage
bmw,132095
mercedes_benz,130177
audi,127524
volkswagen,125768
opel,123988
ford,119807


In [47]:
highest_brand_ds["mean_price"] = mean_prices
highest_brand_ds

,mean_mileage,mean_price
bmw,132095,8974
mercedes_benz,130177,9192
audi,127524,10276
volkswagen,125768,6640
opel,123988,4220
ford,119807,5258


There is no significant difference between the mileages of the used cars for the brands which I dealed with in this project. But the prices of the used car for the brands were very different.There is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage.